# Tensors and Variables

@[Xiaoyu Chen](mailto:c-xy17@tsinghua.org.cn)
@[Chaoming Wang](https://github.com/chaoming0625)

In this section ,we will briefly intrduce two basic and important data structures: tensors and vriables. They form the foundation for mathmatical operations of brain dynamics programming (BDP) in BrainPy.

## Tensors

```{note}
If you have the basic knowledge about [NumPy](https://numpy.org/) (the ``tensor`` here is the same as the ``ndarray`` in NumPy), you can skip this part.
```

### Definition

A tensor is a data structure that organizes algebraic objects in a multidimentional vector space. Simply speaking, in BrainPy, a tensor is a multidimensional array that contains the same type of data, most commonly of the numeric or boolean type. 

To enable tensor operations, users should import the ``brainpy.math`` module:

In [4]:
import brainpy.math as bm

bm.set_platform('cpu')

In [5]:
t1 = bm.array([[[0, 1, 2, 3], [1, 2, 3, 4], [4, 5, 6, 7]], 
               [[0, 0, 0, 0], [-1, 1, -1, 1], [2, -2, 2, -2]]])
t1

JaxArray(DeviceArray([[[ 0,  1,  2,  3],
                       [ 1,  2,  3,  4],
                       [ 4,  5,  6,  7]],
         
                      [[ 0,  0,  0,  0],
                       [-1,  1, -1,  1],
                       [ 2, -2,  2, -2]]], dtype=int32))

Here we create a 3-dimensional tensor with the type of `int32`. Tensors created by ``brainpy.math`` will be stored in ``JaxArray``, for their future operations will be accelerated by just-in-time (JIT) compilation.

In [6]:
t1.shape

(2, 3, 4)

The shape of the tensor refers to the length of each axis, or the size of the array in each dimension. For example, ``t1`` is firstly composed of 2 arrays:

In [9]:
print('t1[0]:\n{}'.format(t1[0]))
print('t1[1]:\n{}'.format(t1[1]))

t1[0]:
[[0 1 2 3]
 [1 2 3 4]
 [4 5 6 7]]
t1[1]:
[[ 0  0  0  0]
 [-1  1 -1  1]
 [ 2 -2  2 -2]]


Therefore, the first axis (dimension) is of length 2, and the first element of ``t1.shape`` is 2. Similarly, the second axis is of length 3, because each sub-array of ``t1`` contains 3 arrays, such as ``t[0, 0]``, ``t[0, 1]``, and ``t[0, 2]``.

In [10]:
print('t1[0, 0]:\n{}'.format(t1[0, 0]))
print('t1[0, 1]:\n{}'.format(t1[0, 1]))
print('t1[0, 2]:\n{}'.format(t1[0, 2]))

t1[0, 0]:
[0 1 2 3]
t1[0, 1]:
[1 2 3 4]
t1[0, 2]:
[4 5 6 7]


And the last dimension is of length 4 because there are 4 elements in each sub-sub-array. Below is the visualization of ``t1`` that helps users to better understand the structure of a tensor.

<center><img src="../_static/tensor_shape.png" width="400 px"></center>

In the [advanced tutorial](../tutorial_math/tensors.ipynb), various operations on tensors will be elaborated. Here we just give a few examples of tensor operations that are commonly used in brain dynamics programming.

### Creating a tensor

In [16]:
t2 = bm.arange(4)
# t2: JaxArray(DeviceArray([0, 1, 2, 3], dtype=int32))

t3 = bm.ones((2, 4)) * 1.5
# t3: JaxArray(DeviceArray([[1.5, 1.5, 1.5, 1.5],
#                           [1.5, 1.5, 1.5, 1.5]], dtype=float32))

### tensor operations

In [ ]:
# indexing and slicing
t3[1]
# DeviceArray([1.5, 1.5, 1.5, 1.5], dtype=float32)

t3[:, 2:]
# DeviceArray([[1.5, 1.5],
#              [1.5, 1.5]], dtype=float32)

In [ ]:
# algebraic operations
t2 + t3[0]
# JaxArray(DeviceArray([1.5, 2.5, 3.5, 4.5], dtype=float32))

t3[0] / t1[0, 1]
# DeviceArray([1.5  , 0.75 , 0.5  , 0.375], dtype=float32)

# broadcasting
t2 + 3
# JaxArray(DeviceArray([3, 4, 5, 6], dtype=int32))

t2 + t3
# JaxArray(DeviceArray([[1.5, 2.5, 3.5, 4.5],
#                       [1.5, 2.5, 3.5, 4.5]], dtype=float32))

In [ ]:
# some functions
bm.dot(t2, t3.T)
# JaxArray(DeviceArray([9., 9.], dtype=float32))

bm.max(t1, axis=2)
# JaxArray(DeviceArray([[3, 4, 7],
#                       [0, 1, 2]], dtype=int32))

t3.flatten()
# JaxArray(DeviceArray([1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5], dtype=float32))

In BrainPy, tensors can be used to store some parameters related to dynamical models. For example, if we define a group of Integrate-and-Fire (LIF) neurons and wish to assign each neuron with a different time constant $\tau$, then we can generate a tensor containing an array of time constants.

In [34]:
n = 6  # assume there are 6 LIF neurons
tau = bm.random.randn(n)*2. + 20.
tau

JaxArray(DeviceArray([19.108969, 19.941414, 18.535835, 17.674583, 22.794453,
                      17.256771], dtype=float32))

Through the code above, a group of time constants is generated from a normal distribution, with a mean of 20 and a variance of 2.

## Variables

We have talked about the definition, operations, and application of tensors in BrainPy. There are some situations, however, where tensors are not applicable. Due to [JIT compilation](jit_compilation.ipynb), once a tensor is given to the JIT compiler, the values inside the tensor cannot be changed. This gives rise to severe limitations, because some properties of the dynamical system, such as the membrane potential, dynamically changes over time. Therefore, we need a new data structure to store such dynamic variables, and that is ``brainpy.math.Variable``.

### ``brainpy.math.Variable``

``brainpy.math.Variable`` is a pointer referring to a tensor. The tensor is stored as its value. The data in a Variable can be changed during JIT compilation. **If a tensor is labeled as a Variable, it means that it is a dynamical variable that changes over time.**

To create or change a tensor into a variable, users just need to wrap the tensor into ``brainpy.math.Variable``:

In [68]:
v = bm.Variable(t2)
v

Variable(DeviceArray([0, 1, 2, 3], dtype=int32))

Note that the array is contained in a "Variable" instead of a "JaxArray".

```{note}
Tensors that are not marked as Variables will be JIT compiled as static data. In [JIT compilation](jit_compilation.ipynb), it is shown that modifications of tensors are invalid in a JIT-compilation environment.
```

Users can access the value in the Variable through its attribute `.value`:

In [69]:
v.value

DeviceArray([0, 1, 2, 3], dtype=int32)

Since the data inside a Variable is a tensor, common operations on tensors can be directly grafted to Variables.

### In-place updating

Though the operations are the same, there are some requirements for updating a Variable. If we directly change a Variable, The returning data will become a tensor but not a Variable.

In [77]:
v2 = v + 2
v2

JaxArray(DeviceArray([5, 6, 7, 8], dtype=int32))

To update the Variable, users are required to use in-place updating, which only modifies the value inside the Variable but does not change the reference pointing to the Variable. In-place updating operations include:

**1\. Indexing and slicing**

  - Indexing: ``v[i] = a``
  - Slicing: ``v[i:j] = b``
  - Slicing the specific values: ``v[[1, 3]] = c``
  - Slicing all values, ``v[:] = d``, ``v[...] = e``

for more details, please refer to [Array Objects Indexing](https://numpy.org/doc/stable/reference/arrays.indexing.html).

In [70]:
v[0] = 10
v[1:3] = 9
v

Variable(DeviceArray([10,  9,  9,  3], dtype=int32))

**2\. Augmented assignment**

- ``+=`` (add)
  - ``-=`` (subtract)
  - ``/=`` (divide)
  - ``*=`` (multiply)
  - ``//=`` (floor divide)
  - ``%=`` (modulo)
  - ``**=`` (power)
  - ``&=`` (and)
  - ``|=`` (or)
  - ``^=`` (xor)
  - ``<<=`` (left shift)
  - ``>>=`` (right shift) 

In [71]:
v -= 3
v <<= 1
v

Variable(DeviceArray([14, 12, 12,  0], dtype=int32))

**3\. ``.value`` assignment**

In [73]:
v.value = bm.arange(4)
v

Variable(DeviceArray([0, 1, 2, 3], dtype=int32))

`` .value`` assignment directly accesses the data stored in the JaxArray. When using `.value`, the new data should be of the same type and shape as the original ones.

In [75]:
try:
    v.value = bm.array([1., 1., 1., 0.])
except Exception as e:
    print(type(e), e)

<class 'brainpy.errors.MathError'> The dtype of the original data is int32, while we got float32.


**4\. ``.update()`` method**

This method will also check if the new data is of the same type and shape as the original ones.

In [76]:
v.update(bm.array([3, 4, 5, 6]))
v

Variable(DeviceArray([3, 4, 5, 6], dtype=int32))

For more details, such as the subtypes of Variables and more information about in-place updating, please see the [advanced tutorial for Variables](../tutorial_math/variables.ipynb).